**Instituto de Informática - UFG**

Disciplina: Inteligência Artificial

Prof. Cedric Luiz de Carvalho

Grupo ( A )

---


Aluna(o): Luana Guedes Barros Martins

---

Aluna(o): Ruan Chaves Rodrigues

---

Aluna(o): Thiago Monteles de Souza

---

# Atividade Prática 05 - Algoritmos Genéticos

Particionamento das atividades:
- Luana Guedes Barros Martins: 
- Ruan Chaves Rodrigues: 
- Thiago Monteles de Souza: Gerente do Projeto

In [1]:
!pip install pyeasyga

In [2]:
# Imports

import pandas as pd
import numpy as np
import seaborn as sns
import re
import random
from operator import mul
from functools import reduce
from pyeasyga import pyeasyga

## Descrição do Problema

Utilizando a base de dados fornecida de alunos do curso de Ciência da Computação da Universidade Federal de Goiás (UFG), devemos determinar qual é o melhor grupo de disciplinas a serem cursadas pelos alunos, a cada semestre, de forma a terem maior sucesso, isto é, terem a maior média geral de notas e o menor número de reprovações. 

Para encontrar a solução do problema, será proposto um modelo de Algoritmo Genético que consiste em uma busca para achar soluções aproximadas. Para a sua implementação, será utilizado a biblioteca [pyeasyga](https://pyeasyga.readthedocs.io/en/latest/).

## Tratamento dos Dados
*Descrição do que foi feito*

In [3]:
"""
# Carregar o dataset "baseDeDados.csv" para o ambiente virtual

from google.colab import files
uploaded = files.upload()
"""

'\n# Carregar o dataset "baseDeDados.csv" para o ambiente virtual\n\nfrom google.colab import files\nuploaded = files.upload()\n'

In [4]:
# Carregando a base de dados
dataset = pd.read_csv('baseDeDados.csv', sep=';')

/home/luana/miniconda3/envs/IA/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def extend_list(lst):
    """
    Função utilitária para o pré-processamento da tabela de dependências.
    Uniformiza o tamanho de sublistas.
    """
    try:
        lst[2]
    except:
        lst.append('')
    return lst

def list_to_int(lst):
    """
    Função utilitária para o pré-processamento da tabela de dependências.
    Gera uma lista de dependências.
    """
    new_lst = []
    for item in lst:
        try:
            new_lst.append(int(item))
        except:
            continue
    return new_lst

def levenshtein(s1, s2):
    """
    Função utilitária para o pré-processamento de tabelas.
    Mede a distância Levenshtein ( distância de edição ) entre duas strings.
    """
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1       
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

def tabela_notas(df):
    """
    Retorna a média geral dos alunos em cada matéria.
    """
    return df[['nome_componente', 'nota_discente_componente']]\
    .groupby('nome_componente')\
    .mean()\
    .to_dict()\
    ['nota_discente_componente']

def tabela_dependencias_grade_nova(df):
    """
    Esta função transforma o texto abaixo, copiado de 
    http://inf.ufg.br/sites/default/files/uploads/Resolucao_CEPEC_2017_1561.pdf ,
    em uma tabela de dependências para a grade nova.
    """
    novos_requisitos = """
    01 Álgebra Linear IME 64 0 64 NC OBR
    02 Algoritmos e Estruturas de Dados 1 INF 26 32 32 64 NC OBR
    03 Algoritmos e Estruturas de Dados 2 INF 2 64 0 64 NC OBR
    04 Análise e Projeto de Algoritmos INF 21, 03 64 0 64 NC OBR
    05 Arquitetura de Computadores INF 21 48 16 64 NC OBR
    06 Banco de Dados INF 29 48 16 64 NC OBR
    07 Cálculo 1A IME 96 0 96 NC OBR
    08 Cálculo 2A IME 7 96 0 96 NE OBR
    09 Cálculo Numérico IME 18 64 0 64 NE OBR
    10 Circuitos Digitais EMC 21 64 0 64 NE OBR
    11 Compiladores INF 03, 28 48 16 64 NE OBR
    12 Computação e Sociedade INF 32 0 32 NC OBR
    13 Computação Gráfica INF 22 32 32 64 NE OBR
    14 Computação Paralela INF 39 48 16 64 NE OBR
    15 Eletrônica para Computação EMC 20 96 0 96 NE OBR
    16 Engenharia de Requisitos INF 16 48 64 NE OBR
    17 Engenharia de Software INF 64 0 64 NC OBR
    18 Equações Diferenciais Ordinárias IME 01, 08 64 0 64 NE OBR
    19 Física I IF 7 64 0 64 NE OBR
    20 Física III IF 19 64 0 64 NE OBR
    21 Fundamentos de Matemática para Computação INF 64 0 64 NC OBR
    22 Geometria Analítica IME 64 0 64 NE OBR
    23 Inteligência Artificial INF 02, 29, 33 64 0 64 NE OBR
    24 Interação Humano-Computador INF 02, 33 32 32 64 NC OBR
    25 Introdução à Computabilidade e à Complexidade Computacional INF 04, 28 64 0 64 NE OBR
    26 Introdução à Programação INF 48 80 128 NC OBR
    27 Linguagens e Paradigmas de Programação INF 26 32 32 64 NC OBR
    28 Linguagens Formais e Autômatos INF 21 64 0 64 NE OBR
    29 Lógica Matemática INF 21 64 0 64 NC OBR
    30 Matemática Discreta INF 21 64 0 64 NE OBR
    31 Metodologia de Pesquisa em Computação INF 32 0 32 NE OBR
    32 Pesquisa Operacional INF 1 64 0 64 NE OBR
    33 Probabilidade e Estatística A IME 7 64 0 64 NC OBR
    34 Programação Orientada a Objetos INF 26 32 32 64 NC OBR
    35 Projeto de Software INF 17, 34 64 0 64 NC OBR
    36 Projeto Final de Curso 1 INF 0 64 64 NE OBR
    37 Projeto Final de Curso 2 INF 36 0 64 64 NE OBR
    38 Redes de Computadores 1 INF 48 16 64 NE OBR
    39 Sistemas Distribuídos INF 38 48 16 64 NE OBR
    40 Sistemas Gerenciadores de Banco de Dados INF 6 64 0 64 NE OBR
    41 Sistemas Operacionais INF 02, 05 64 0 64 NE OBR
    42 Software Básico INF 2 0 64 64 NE OBR
    43 Teoria dos Grafos INF 30 64 0 64 NE OBR
    44 Aprendizagem de Máquina e Mineração de Dados INF 23 32 32 64 NE OPT
    45 Armazém de Dados INF 40 48 16 64 NE OPT
    46 Avaliação de Desempenho de Sistemas Computacionais INF 33 64 0 64 NE OPT
    47 Banco de Dados Distribuídos INF 40 64 0 64 NE OPT
    48 Computação Evolucionária INF 32 32 64 NE OPT
    49 Computação Móvel e Ubíqua INF 34, 35 16 48 64 NE OPT
    50 Computação Verde INF 38 32 32 64 NE OPT
    51 Desenvolvimento de Sistemas para Web INF 34 32 32 64 NE OPT
    52 Desenvolvimento de Software Dirigido por Modelos INF 02, 34 32 32 64 NE OPT
    53 Empreendedorismo Digital INF 32 32 64 NE OPT
    54 Engenharia de Sistemas INF 64 0 64 NE OPT
    55 Engenharia de Software Baseada em Busca INF 17 32 32 64 NE OPT
    56 Fundamentos da Programação Lógica e Funcional INF 27, 29 64 0 64 NE OPT
    57 Integração de Dados INF 40 64 0 64 NE OPT
    58 Introdução à Língua Brasileira de Sinais FL 64 0 64 NE OPT
    59 Introdução à Teoria das Provas INF 29 64 0 64 NE OPT
    60 Introdução à Teoria dos Tipos INF 29 64 0 64 NE OPT
    61 Jogos Digitais INF 34 16 48 64 NE OPT
    62 Meta-heurísticas INF 2 64 0 64 NE OPT
    63 Metodologia e Experimentação em Engenharia de Software INF 64 0 64 NE OPT
    64 Microcontroladores INF 10, 26 32 32 64 NE OPT
    65 Processamento Digital de Imagens INF 01, 26 64 0 64 NE OPT
    66 Recuperação de Informação INF 02, 33 64 0 64 NE OPT
    67 Redes de Computadores 2 INF 38 32 32 64 NE OPT
    68 Redes Neuronais Artificiais INF 23 32 32 64 NE OPT
    69 Segurança e Auditoria de Sistemas INF 38 32 32 64 NE OPT
    70 Sistemas Multiagentes INF 34 32 32 64 NE OPT
    71 Tópicos Avançados em Redes Sem Fio INF 16 48 64 NE OPT
    72 Tópicos em Computação INF 64 0 64 NE OPT
    73 Tópicos em Teste de Software INF 48 16 64 NE OPT
    74 Verificação Formal de Programas INF 29 32 32 64 NE OPT
    75 Visão Robótica INF 01, 65 64 0 64 NE OPT
    76 Visualização de Informações INF 3 32 32 64 NE OPT
    77 Web Semântica INF 34 32 32 64 NE OPT
    """.split('\n')[1:-1]

    novos_requisitos = [ re.sub('[0-9]* [0-9]* [0-9]* [A-Z]* [A-Z]*$', '', line) for line in novos_requisitos ]

    novos_requisitos = [ re.split('(^[0-9]* )', line)[1:] for line in novos_requisitos ]

    novos_requisitos = [ [x[0]] + re.split(' [A-Z][A-Z][A-Z]? ', x[1]) for x in novos_requisitos ]

    novos_requisitos = [ extend_list(x) for x in novos_requisitos ]

    novos_requisitos = [ x[1:] for x in novos_requisitos ]
    
    for idx, item in enumerate(novos_requisitos):
        tmp = []
        p1 = item[0]
        p2 = item[1]
        p1 = p1.lstrip()
        p1 = p1.split(' ', 1)
        tmp.extend(p1)
        tmp.append(p2)
        if len(tmp) == 2:
            tmp.append([])
        novos_requisitos[idx] = tmp
              
    novos_requisitos = [
        {
            "key": int(x[0]),
            "name": x[1],
            "deps": list_to_int(x[2].split(','))
        } for x in novos_requisitos
    ]
    
    materias_tabela = df[df['codigo_matriz_curricular']=='CICOMP-BI-3']['nome_componente'].drop_duplicates().values

    grafo_requisitos = []
    for x in materias_tabela:
        distances = []
        names = []
        for y in novos_requisitos:
            distances.append( levenshtein(x.lower(),y['name'].lower()) )
            names.append(y['name'])
        match = names[distances.index(min(distances))]
        for y in novos_requisitos:
            if y['name'] == match:
                y['name'] = x
                grafo_requisitos.append(y)

    for item in grafo_requisitos:
        deps = item['deps']
        for idx,dep in enumerate(deps):
            for item2 in grafo_requisitos:
                if item2['key'] == dep:
                    item['deps'][idx] = item2['name']

    dependencias_grade_nova = []
    for item in grafo_requisitos:
        for dep in item['deps']:
            row = {
                "materia": item['name'],
                "dependencia": dep
            }
            dependencias_grade_nova.append(row)
    dependencias_grade_nova = pd.DataFrame(dependencias_grade_nova)
    return dependencias_grade_nova

def tabela_dependencias_grade_antiga(df):
    """
    Esta função transforma o dicionário abaixo, elaborado a partir de 
    http://www.inf.ufg.br/sites/default/files/ProjetoPedagogico2008.pdf ,
    em uma tabela de dependências para a grade nova.
    """
    requisitos = [
        {
        "materia": "Programação de Computadores 2",
        "dependencia": "Programação de Computadores 1"
    },
        {
        "materia": "Estrutura de Dados I",
        "dependencia": "Programação de Computadores 1"
    },
        {
        "materia": "Programação Orientada a Objetos",
        "dependencia": "Programação de Computadores 1"
    },
        {
        "materia": "Linguagens de Programação",
        "dependencia": "Programação de Computadores 1"
    },
        {
        "materia": "Estruturas de Dados II",
        "dependencia": "Estruturas de Dados I"
    },
        {
        "materia": "Análise e Projeto de Algoritmos",
        "dependencia": "Estruturas de Dados I"
    },
        {
        "materia": "Teoria da Computação",
        "dependencia": "Linguagens Formais e Autômatos"
    },
        {
        "materia": "Compiladores",
        "dependencia": "Linguagens Formais e Autômatos"
    },    
        {
        "materia": "Compiladores",
        "dependencia": "Estruturas de Dados II"
    },
        {
        "materia": "Redes de Computadores 2",
        "dependencia": "Redes de Computadores 1"
    },
        {
        "materia": "Sistemas Distribuídos",
        "dependencia": "Redes de Computadores 1"
    },
        {
        "materia": "Inteligência Artificial",
        "dependencia": "Lógica Matemática"
    },
        {
        "materia": "Banco de Dados",
        "dependencia": "Lógica Matemática"
    },
        {
        "materia": "Sistemas Gerenciadores de BD",
        "dependencia": "Banco de Dados"
    },
        {
        "materia": "Sistemas Operacionais 2",
        "dependencia": "Sistemas Operacionais 1"
    },
        {
        "materia": "Cálculo 2",
        "dependencia": "Cálculo 1"
    },
        {
        "materia": "Projeto Final de Curso 2",
        "dependencia": "Projeto Final de Curso 1"
    }
    ]


    materias_tabela = df[df['codigo_matriz_curricular']=='CICOMP-BI-2']['nome_componente'].drop_duplicates().values

    for idx, req in enumerate(requisitos):
        for key, value in req.items():
            tmp = []
            for materia in materias_tabela:
                lev = levenshtein(value.lower(), materia.lower())
                if lev < 10:
                    tmp.append( [ lev, materia ] )
            tmp = sorted(tmp, key=lambda x: x[0])
            try:
                new_value = tmp[0][1]
                requisitos[idx][key] = new_value
            except:
                pass

    dependencias_grade_antiga = pd.DataFrame(requisitos)
    return dependencias_grade_antiga

def tabela_equivalencias(df):
    """
    Sugere uma tabela de equivalências entre matérias.
    """
    materias_novas = df[df['codigo_matriz_curricular']=='CICOMP-BI-3']['nome_componente'].drop_duplicates().values
    materias_antigas = df[df['codigo_matriz_curricular']=='CICOMP-BI-2']['nome_componente'].drop_duplicates().values

    equivalencias = []
    for mn in materias_novas:
        tmp = []
        for ma in materias_antigas:
            lev = levenshtein(mn, ma)
            if lev < 10:
                tmp.append( [levenshtein(mn, ma), ma] )
                tmp = sorted(tmp, key=lambda x: x[0])
        if not tmp:
            continue
        match = tmp[0][1]
        row = {
                "nova": mn,
                "antiga": match
            }
        equivalencias.append(row)
    equivalencias = \
    [{'nova': 'CÁLCULO 1A', 'antiga': 'CÁLCULO 1'},
     {'nova': 'CÁLCULO 2A', 'antiga': 'CÁLCULO 2'},
    #  {'nova': 'FÍSICA III', 'antiga': 'TÓPICOS 1'},
     {'nova': 'ÁLGEBRA LINEAR', 'antiga': 'ÁLGEBRA LINEAR'},
     {'nova': 'PROBABILIDADE E ESTATÍSTICA A',
      'antiga': 'PROBABILIDADE E ESTATÍSTICA'},
    #  {'nova': 'MATEMÁTICA DISCRETA', 'antiga': 'MATEMÁTICA DISCRETA'},
     {'nova': 'PROGRAMAÇÃO ORIENTADA A OBJETOS',
      'antiga': 'PROGRAMAÇÃO ORIENTADA A OBJETOS'},
     {'nova': 'ARQUITETURA DE COMPUTADORES',
      'antiga': 'ARQUITETURA DE COMPUTADORES'},
     {'nova': 'ANÁLISE E PROJETO DE ALGORITMOS',
      'antiga': 'ANÁLISE E PROJETO DE ALGORITMOS'},
     {'nova': 'BANCO DE DADOS', 'antiga': 'BANCO DE DADOS'},
     {'nova': 'COMPUTAÇÃO E SOCIEDADE', 'antiga': 'COMPUTADOR E SOCIEDADE'},
    #  {'nova': 'FÍSICA I', 'antiga': 'TÓPICOS 1'},
     {'nova': 'LÓGICA MATEMÁTICA', 'antiga': 'LÓGICA MATEMÁTICA'},
     {'nova': 'ENGENHARIA DE SOFTWARE', 'antiga': 'ENGENHARIA DE SOFTWARE'},
     {'nova': 'INTRODUÇÃO À PROGRAMAÇÃO', 'antiga': 'INTRODUÇÃO À COMPUTAÇÃO'}]

    manual = [
        {'nova': 'ALGORITMOS E ESTRUTURAS DE DADOS 1', 'antiga': 'ESTRUTURAS DE DADOS I'},
        {'nova': 'ALGORITMOS E ESTRUTURAS DE DADOS 2', 'antiga': 'ESTRUTURAS DE DADOS II'}]
    equivalencias += manual
    equivalencias = pd.DataFrame(equivalencias)
    return equivalencias


In [6]:
class Aluno(object):
    """
    Objeto que representa um aluno hipotético, gerado automaticamente.
    """
    def __init__(self, df, grade_nova=False):
        self.df = df
        self.grade_nova = grade_nova
        self.equivalencias = tabela_equivalencias(df)
        self.deps_antiga = tabela_dependencias_grade_antiga(df)
        self.deps_nova = tabela_dependencias_grade_nova(df)
        self.notas = tabela_notas(df)
        
    def fluxo(self, periodo):
        """
        Retorna as matérias previstas para um período pelo fluxo.
        """
        if self.grade_nova:
            idx = '3'
        else:
            idx = '2'
        tmp = self.df[self.df['codigo_matriz_curricular']== 'CICOMP-BI-' + idx]\
        [['nome_componente', 'semestre_previsto_componente']]
        tmp = tmp[tmp['semestre_previsto_componente'] == periodo]\
        [['nome_componente']]\
        .drop_duplicates().values.flatten().tolist()
        return tmp
    
    def expandir_materias(self, materias):
        """
        Dado uma lista de matérias da grade nova, expandir para as equivalentes na grade antiga,
        e vice-versa.
        """
        tmp = materias[::]
        dct = self.equivalencias.to_dict()
        for key in dct.keys():
            dct[key] = { v:k for k,v in dct[key].items() }
        for materia in materias:
            try:
                key = 'nova'
                other = 'antiga'
                dct[key] = { v:k for k,v in dct[key].items() }
                tmp.append(dct[other][dct[key][materia]])
            except:
                pass
            try:
                key = 'antiga'
                other = 'nova'
                dct[key] = { v:k for k,v in dct[key].items() }
                tmp.append(dct[other][dct[key][materia]])            
            except:
                pass            
        return sorted(list(set(tmp)))
    
    def checar_dependencias(self, materia, cursadas):
        """
        Retorna um par de valores booleanos. 
        
        O primeiro diz se as matérias no parâmetro "cursadas"
        satisfazem aos pré-requisitos da matéria no parâmetro "materia"
        na grade antiga.
        
        O segundo diz se as matérias no parâmetro "cursadas"
        satisfazem aos pré-requisitos da matéria no parâmetro "materia"
        na grade nova.        
        """
        try:
            req_antiga = self.deps_antiga[self.deps_antiga['materia']==materia]\
            .to_dict()['dependencia']\
            .values()
        except:
            req_antiga = []
        try:
            req_nova = self.deps_nova[self.deps_nova['materia']==materia]\
            .to_dict()['dependencia']\
            .values()
        except:
            req_nova = []
        check_antiga = len(set(req_antiga).intersection(set(cursadas))) == len(set(req_antiga))
        check_nova = len(set(req_nova).intersection(set(cursadas))) == len(set(req_nova))
        return check_antiga, check_nova
    
    def criar_fluxo(self, periodos=8, lim=6):
        """
        Gera um aluno hipotético que sempre segue o fluxo.
        Retorna as matérias cursadas pelo aluno até o n-ésimo período,
        obedecendo um limite de matérias por período.
        """
        materias = []
        pendente = []
        materias.extend([ (m,1) for m in self.fluxo(1) ])
        for p in range(2,periodos+1):
            f = self.fluxo(p)
            tmp = pendente[::]
            for item in f:
                tmp.append( [item, self.notas[item], 'not_pending'] )
            tmp = sorted(tmp, key=lambda x: x[1], reverse=True)
            
            for idx,item in enumerate(tmp):
                m = [x[0] for x in materias]
                status_1, status_2 = self.checar_dependencias(item, m)
                if self.grade_nova == False and status_1 == False:
                    tmp[idx][2] = 'blocked'
                if self.grade_nova == True and status_2 == False:
                    tmp[idx][2] = 'blocked'
                if self.grade_nova == False and status_1 == True:
                    tmp[idx][2] = 'not_pending'
                if self.grade_nova == True and status_2 == True:
                    tmp[idx][2] = 'not_pending'    
                    
            blocked = [x for x in tmp if x[2] == 'blocked' ]
            unblocked = [x for x in tmp if x[2] != 'blocked']
            not_pending = unblocked[0:lim]
            pending = unblocked[lim:len(unblocked)]
            for idx,item in enumerate(pending):
                pending[idx][2] = 'pending'
            
            pendente = blocked + pending
            materias.extend([ (m[0],p) for m in not_pending ])
        return materias
    
    def gerar_notas(self, m):
        """
        Retorna a média geral associada à matéria m.
        """
        return self.notas[m]
    
    def gerar_entrada(self, materias, grade_nova=False, grade_antiga=True, expand=True):
        """
        A partir de uma lista de matérias cursadas,
        gera uma lista de matérias possíveis, 
        que servirão como entrada para o algoritmo genético.
        """
        entrada = []
        if expand:
            atuais = self.expandir_materias(materias)
        else:
            atuais = materias[::]
        materias = []
        materias_novas = []
        materias_antigas = []
        if grade_nova:
            materias_novas = self.df[self.df['codigo_matriz_curricular']== 'CICOMP-BI-3']\
            [['nome_componente']].drop_duplicates().values.flatten().tolist()
        if grade_antiga:
            materias_antigas = self.df[self.df['codigo_matriz_curricular']== 'CICOMP-BI-2']\
            [['nome_componente']].drop_duplicates().values.flatten().tolist()
        materias = materias_novas + materias_antigas
        for m in materias:
            status_1, status_2 = self.checar_dependencias(m, atuais)
            if grade_nova == True and status_2 == True and m not in atuais:
                entrada.append(m)
            if grade_nova == False and status_1 == True and m not in atuais:
                entrada.append(m)
            
        notas = [ self.gerar_notas(x) for x in entrada ]
        return pd.DataFrame(list(zip(entrada, notas))).rename(columns={
            0: 'materia',
            1: 'media_geral'
        })

## Definição do Modelo

Afim de buscar uma solução para o problema proposto, temos que a ideia é sugerir um semestre para o aluno que seja o mais balanceado o possível em relação ao seu nível de dificuldade afim de que o aluno possa maximizar seu desempenho naquele semestre. Assim, o aluno não terá problemas de pegar variás matérias difíceis ao mesmo tempo em algum ponto do curso.

Partindo de uma lista inicial de matérias já cursadas, consultando as tabelas de dependências e equivalências, é produzido uma lista com as matérias possíveis de serem cursadas pelo aluno no próximo semestre. Nesta lista de matérias possíveis, cada matéria está acompanhada pela média geral de todos os alunos nesta matéria. 

### Entrada do algoritmo

A entrada do algoritmo consistirá em um conjunto de matérias permitidas, e, associado a cada uma delas, uma média geral de todos os seus alunos. Esse valor consistirá na definição de "nível de dificuldade" daquela matéria.

In [7]:
"""
Utilizando a classe Aluno, que representa um aluno hipotético, iremos
criar um aluno que seguiu o fluxo nos dois primeiros semestres, com 5 
matéria em cada semestre
"""

aluno = Aluno(dataset)
NUMERO_DE_MATERIAS = 5
PERIODO = 2

materias = [x[0] for x in aluno.criar_fluxo(periodos=PERIODO, lim=NUMERO_DE_MATERIAS)]
materias

['CÁLCULO 1',
 'LÓGICA MATEMÁTICA',
 'MATEMÁTICA DISCRETA',
 'INTRODUÇÃO À COMPUTAÇÃO',
 'PROGRAMAÇÃO DE COMPUTADORES 1 CN',
 'PROGRAMAÇÃO DE COMPUTADORES 2 CN',
 'SISTEMAS DIGITAIS',
 'FÍSICA PARA COMPUTAÇÃO',
 'CÁLCULO 2',
 'ÁLGEBRA LINEAR']

### Representação do Cromossomo

Cada cromossomo terá $k$ genes, o mesmo tamanho da entrada, onde cada gene irá corresponder a uma matéria do curso de Ciência da Computação. Cada gene possuirá um valor binário que irá indicar se a matéria vai ser incluída na indicação ou não.

In [8]:
"""
Criação da representação de uma solução candidata (cromossomo).
Gera um cromossomo com valores binários com k genes, sendo k o 
tamanho da entrada.
"""

def create_individual(data):
    return [random.randint(0,1) for _ in range(len(data))]

In [9]:
# Numero de cromossomos por população
N_POPULACAO = 50

Dado que a entrada tem tamanho $k$, teremos uma população inicial de cromossomos aleatórios.

### Método de seleção

Nesta fase os indivíduos mais aptos da geração atual são selecionados. Esses indivíduos são utilizados para gerar uma nova população por cruzamento. Cada indivíduo tem uma probabilidade de ser selecionado proporcional à sua aptidão. 

*TODO*

In [10]:
def selection(population):
    return random.choice(population)

### Cruzamento e Estratégia de Corte

Os indivíduos selecionados na seleção serão cruzados da seguinte forma: a lista de indivíduos selecionados é embaralhada aleatoriamente criando-se, desta forma, uma segunda lista, chamada lista de parceiros. Cada indivíduo selecionado é então cruzado com o indivíduo que ocupa a mesma posição na lista de parceiros. Os cromossomos de cada par de indivíduos a serem cruzados são particionados em um ponto, chamado ponto de corte, sorteado aleatoriamente. Um novo cromossomo é gerado permutando-se a metade inicial de um cromossomo coma metade final do outro. 

In [11]:
"""
Para a função de cruzamento, será necessário dois indivíduos (pais)
para a geração de indíviduos-filhos. Para isso o ponto de corte será
escolhido aleatóriamente.
"""

def crossover(parent1, parent2):
    child1 = []
    child2 = []
    
    # Posição de um gene na cadeia para ser usado como ponto de corte
    locus = random.randrange(1, len(parent1))
        
    for i in range(len(parent1)):
        if i < locus:
            child1.append(parent1[i])
            child2.append(parent2[i])
        else: 
            child1.append(parent2[i])
            child2.append(parent1[i])
            
    return child1, child2

In [12]:
# Probabilidade de cruzamento
C_PROB = 0.8

### Mutação

A operação de mutação é utilizada para garantir uma maior varredura do espaço de estados e evitar convergências para mínimos/máximos locais. A mutação é efeutada alterando o valor de um gene de um indivíduo aleatório com uma determinada probabilidade.

In [13]:
"""
Para a função de mutação, será escolhido um indivíduo aleatoriamente 
na lista de individuos para sofrer a mutação.
"""
def mutate(individuals):
    # seleciona um individuo 
    idx = random.randrange(len(individuals))
    
    if individuals[idx] == 0:
        individuals[idx] = 1
    else:
        individuals[idx] = 0

In [14]:
# Probabilidade de mutação
M_PROB = 0.05

### Critério de parada

O critério de parada irá determinar quando chegaremos ao fim da execução do algoritmo genético. Foi escolhido o método de parada que consiste em estabelecer um número máximo de gerações.

In [15]:
# Quantidade de gerações
N_GEN = 20

### Função de aptidão (fitness)

Seja $z$ a quantidade de matérias que o aluno deseja cursar no próximo semestre.

Definidas as variáveis:  

$$ A = \left| \frac{ \sum_{i=1}^{k} m_{i} - \sum_{i=1}^{k} (m_{i} * b_{i}) }{k} \right|$$

$$ B = \left| 2^{z} - \prod_{i=1}^{k} 2^{b_{i}} \right|$$

Então queremos selecionar, a cada geração do algoritmo, os cromossomos que maximizam a seguinte função de aptidão:  


$$ f(b_{1}, ... , b_{k}) = \frac{1}{A + 1} * \frac{1}{B + 1}$$

O melhor cromossomo possível é aquele que obtém $A = 0$ e $B = 0$, alcançando assim $f(b_{1}, ... , b_{k}) = 1$.

$A = 0$ significa que a média das médias na seleção do cromossomo é exatamente igual à média das médias de todas as matérias possíveis. A função de $A$ é penalizar cromossomos que sugerem seleções desequilibradas de matérias, onde a média das médias da seleção se distancia da média das médias de todas as matérias possíveis.

$B = 0$ significa que o cromossomo sugeriu exatamente $z$ matérias. A função de $B$ é penalizar cromossomos que sugerem uma quantidade de matérias a serem cursadas distinta da quantidade de matérias $z$ que o aluno deseja cursar. 

Na variável $B$, o número 2 foi escolhido como base dos expoentes, o qual comprovamos empiricamente ser eficiente para a distinção entre cromossomos com quantidades corretas e incorretas de sugestões. Manter a base dos expoentes em um valor baixo também garante a eficiência computacional do cálculo da função de aptidão.

In [16]:
"""
A função de aptidão deve receber como paramêtros: uma representação de 
uma possível solução, e os dados que serão usados para determinar a 
avaliação de um indivíduo.
"""

def fitness (individual, data):
    N = [ x[1] for x in data ]
    selected = [ N[i] for i,v in enumerate(individual) if v == 1 ]
    first_term = 1 / ( np.absolute( np.mean(N) - np.mean(selected)) + 1 )
    
    selected = [ 2**(v) for i,v in enumerate(individual) if v == 1 ]
    second_term = 1 / ( np.absolute(2**(NUMERO_DE_MATERIAS) - reduce(mul,selected,1)) + 1 )
    fitness = first_term * second_term
    
    return fitness

## Execução e Resultados

In [17]:
# Mostra as matérias que estão sendo sugeridas pelo cromossomo

def leitura(individual, data):
    return [ data[i][0] for i,v in enumerate(individual[1]) if v == 1 ]

In [18]:
"""
Inicializa a classe GenecticAlgorithm e define todas as funções utilizadas
pelo algoritmo genético.
"""

# Constantes definidas posteriormente
print("Tamanho da população: ", N_POPULACAO)
print("Número de gerações: ", N_GEN)
print("Probabilidade de crossover: ", C_PROB)
print("Probabilidade de mutação: ", M_PROB)

def algoritmo_genetico(data):
    ga = pyeasyga.GeneticAlgorithm(data,
                               population_size=N_POPULACAO,
                               generations=N_GEN,
                               crossover_probability=C_PROB,
                               mutation_probability=M_PROB,
                               elitism=True,
                               maximise_fitness=True)

    # Definição do cromossomo
    ga.create_individual = create_individual
        
    # Função de crossover
    # ga.crossover_function = crossover
    
    # Função de mutação
    # ga.mutate_function = mutate
    
    # Função de seleção
    # ga.selection_function = selection
    
    # Função de aptidão
    ga.fitness_function = fitness

    # Executa o algoritmo genético
    ga.run()
    
    return leitura(ga.best_individual(), data)

Tamanho da população:  50
Número de gerações:  20
Probabilidade de crossover:  0.8
Probabilidade de mutação:  0.05


In [19]:
def gerar_plano(aluno, materias, grade_nova=False, grade_antiga=True, expand=True):
    while True:
        try:
            entrada = aluno.gerar_entrada(materias, grade_nova=grade_nova, grade_antiga=grade_antiga, expand=expand)
            data = [tuple(x) for x in entrada.values.tolist()]
            ag = algoritmo_genetico(data)
            if ag:
                return ag
            materias = materias + ag
        except:
            break

In [20]:
curso = []
for item in gerar_plano(aluno, materias):
    curso.append(item)
    
curso

['PROGRAMAÇÃO ORIENTADA A OBJETOS',
 'ENGENHARIA DE SOFTWARE',
 'LINGUAGENS FORMAIS E AUTÔMATOS',
 'PROJETO DE SOFTWARE',
 'TÓPICOS 1']

In [21]:
# Função utilitária de uniformização de tamanho de arrays
def uniform_length(labels):
    max_len = max([ len(x) for x in labels ])
    for idx,item in enumerate(labels):
        while len(item) != max_len:
            labels[idx].append(0.0)
    return labels

In [22]:
labels = curso

scores = np.array( [ sorted([ (aluno.gerar_notas(y), y) for y in x ], key=lambda x: x[0]) for x in curso ])

labels = [ [ x[1] for x in y ] for y in scores ]
labels = uniform_length(labels)
scores = np.array([ np.array([ x[0] for x in y ]) for y in scores ])
flat = np.concatenate(scores).ravel().astype(np.float)
min_val = np.min(flat)
max_val = np.max(flat)
scores = [ [ 1 - (( x - min_val ) / ( max_val - min_val )) for x in y ] for y in scores ]
scores = [ [ round(x,2) for x in y ] for y in scores ]
scores = uniform_length(scores)
scores = scores[::-1]

KeyError: 'P'

In [ ]:
dataplot = pd.DataFrame(labels)
dataplot.index = dataplot.index.map(lambda x: str(x + 1 + PERIODO) + "º período")
dataplot